In [1]:
import pandas as pd
import re
import json
import xml.etree.ElementTree as xet
import requests as req
import time
import glob
import csv
import regex
import tqdm
import pytz
import io
import numpy as np
from bs4 import BeautifulSoup

In [2]:
def parse_sitemap(fn):
    #page = io.StringIO(req.get(fn).text)

    xml = xet.parse(fn)
    xml_root = xml.getroot()
    news_list = []
    
    for news in xml_root:

        url = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}loc').text
        lastmod = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}lastmod')
        if hasattr(lastmod, 'text'):
            lastmod = lastmod.text
        else:
            lastmod = ''
        
        news_list.append(dict(zip(['url', 'lastmod'], [url, lastmod])))

    return news_list

In [3]:
sitemap_urls = parse_sitemap(io.StringIO(req.get('https://tvrain.ru/sitemap/xml/sitemap.xml').content.decode()))
sitemap_urls[:5]

[{'url': 'https://tvrain.ru/sitemap/xml/story_part0.xml',
  'lastmod': '2020-12-13'},
 {'url': 'https://tvrain.ru/sitemap/xml/articles_part0.xml',
  'lastmod': '2020-12-13'},
 {'url': 'https://tvrain.ru/sitemap/xml/tags_part0.xml',
  'lastmod': '2020-12-13'},
 {'url': 'https://tvrain.ru/sitemap/xml/tree_part0.xml',
  'lastmod': '2020-12-13'},
 {'url': 'https://tvrain.ru/sitemap/xml/teleshow_part0.xml',
  'lastmod': '2020-12-13'}]

In [4]:
news_db = []

for fn in tqdm.tqdm_notebook(sitemap_urls):
    s1 = time.time()

    try:
        text = io.StringIO(req.get(fn['url']).content.decode())
        news = parse_sitemap(text)
    except Exception as e:
        print(e)
        continue
        
    news_db += news
    s2 = time.time()
    time.sleep(max(1 - s2 + s1, 0))

<ipython-input-4-5701dbbfbec7>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for fn in tqdm.tqdm_notebook(sitemap_urls):


In [5]:
df = pd.DataFrame(news_db)
df['lastmod'] = pd.to_datetime(df['lastmod'], format='%Y-%m-%dT%H:%M:%S', utc=True)
df.head()

,url,lastmod
0,https://tvrain.ru/story/delo_pussy_riot/,2020-11-24 14:55:45+00:00
1,https://tvrain.ru/story/bolotnoe_delo/,2020-11-10 10:56:25+00:00
2,https://tvrain.ru/story/situatsija_v_ukraine/,2020-11-05 13:47:14+00:00
3,https://tvrain.ru/story/prisoedinenie_kryma_k_...,2020-11-21 18:06:32+00:00
4,https://tvrain.ru/story/ukrainskij_krizis/,2020-11-05 13:47:14+00:00


In [6]:
df[(df.lastmod >= '2019-11-01')].to_json('news_tvrain.json', date_unit='s', date_format='iso')
df[(df.lastmod >= '2019-11-01')].shape

(4049, 2)

# Parsing

In [16]:
df = pd.read_json('db/news_tvrain.json')
df.head()

,lastmod,url,downloaded
0,1529686680000,https://russian.rt.com/sport/article/527234-br...,False
1,1529662620000,https://russian.rt.com/sport/news/526949-smoln...,False
10,1529669520000,https://russian.rt.com/world/news/527000-smi-k...,False
100,1529701260000,https://russian.rt.com/sport/article/527375-it...,False
1000,1529982780000,https://russian.rt.com/world/news/529049-pompe...,False


In [20]:
def parse_rain_news(text):
    soup = BeautifulSoup(text, features='lxml')
    
    news_text = ''
    lead = soup.find(name='div', attrs={'class': 'document-lead'})
    
    if lead:
        lead_ps = lead.find_all(name='p')
        news_text += " ".join([lead_p.get_text(" ", strip=True) for lead_p in lead_ps])
        
    article = soup.find(name="div", attrs={'class': "article-full__text"})

    if article:
        article_ps = article.find_all(name='p')
        news_text += " ".join([article_p.get_text(" ", strip=True) for article_p in article_ps])
        news_text = re.sub(r'[\n\r]+', ' ', news_text).replace(u'\xa0', u' ')
    else:
        news_text = ''

    tags_elem = soup.find(name="meta", attrs={'name': "keywords"})
    tags = re.sub(r'[\n\r]+', ' ', tags_elem.get('content')).replace(u'\xa0', u' ') if tags_elem else ''
    
    description_elem = soup.find(name="meta", attrs={'name': "description"})
    description = re.sub(r'[\n\r]+', ' ', description_elem.get('content')).replace(u'\xa0', u' ') if description_elem else ''
    
    return {'news_text': news_text, 'tags': tags, 'description': description}

In [21]:
parse_rain_news(req.get('https://tvrain.ru/news/po_delu_ob_ubijstve_spetsnazovtsa_gru_zaderzhali_vosmogo_podozrevaemogo-487272/').text)

{'news_text': 'Следственный комитет задержал восьмого подозреваемого по делу об убийстве бывшего спецназовца ГРУ Никиты Белянкина. Об этом сообщается на сайте СК.Сейчас с ним проводятся следственные действия. Имя задержанного не сообщается. В СК также сообщили, что остальные семь фигурантов дела арестованы. Бывший спецназовец ГРУ Никита Белянкин, участвовавший в боевых действий в Сирии, был убит 2 июня в деревне Путилково Красногорского района Подмосковья. Белянкина ударили ножом в грудь после того, как он заступился за двух мужчин, которых избивали несколько человек. Заведено уголовное дело об убийстве. По данным СМИ, всего в драке участвовали десять человек. ',
 'tags': 'телеканал Дождь, дождь, официальный сайт, новости, информация, независимое тв, прямой эфир, смотреть видео, эфир, онлайн, видео онлайн, программы канала, ведущие дождя, архив, Синдеева, Наталья Синдеева, Михаил Зыгарь, программа передач',
 'description': 'Следственный комитет задержал восьмого подозреваемого по делу 